In [1]:
classifier.eval()

correct = 0
total = 0
for images, labels in test_loader:
    images, labels = images.cuda(), labels.cuda()
    preds = classifier(images).argmax(1)
    correct += (preds == labels).sum().item()
    total += labels.size(0)

print("Clean accuracy:", correct / total)


NameError: name 'classifier' is not defined

In [ ]:
from attacks.adv_patch_pgd import TargetedAdversarialPatchPGD

STOP_CLASS = 14   # ⚠️ confirm STOP index in your dataset

patch_attack = TargetedAdversarialPatchPGD(
    model=classifier,
    target_class=STOP_CLASS,
    patch_size=40,
    steps=300
)

images, _ = next(iter(train_loader))
images = images.cuda()

patch = patch_attack.generate(images)
torch.save(patch, "patch_stop.pt")


In [ ]:
from data.generate_patched_gtsrb import generate_dataset

patch = torch.load("patch_stop.pt").cuda()

patched_images, patch_masks = generate_dataset(
    train_loader,
    patch,
    device="cuda"
)

torch.save(patched_images, "patched_images.pt")
torch.save(patch_masks, "patch_masks.pt")


In [ ]:
from train_segmenter import train_segmenter

images = torch.load("patched_images.pt")
masks = torch.load("patch_masks.pt")

segmenter = train_segmenter(images, masks, epochs=15)

torch.save(segmenter.state_dict(), "segmenter.pt")


In [ ]:
from defense.patch_defense import defend_image
from defense.unet import UNet

segmenter = UNet().cuda()
segmenter.load_state_dict(torch.load("segmenter.pt"))
segmenter.eval()

correct = 0
total = 0

for images, labels in test_loader:
    images, labels = images.cuda(), labels.cuda()

    for img, label in zip(images, labels):
        clean_img = defend_image(img, segmenter)
        pred = classifier(clean_img.unsqueeze(0)).argmax(1)

        correct += (pred == label).item()
        total += 1

print("Defended accuracy:", correct / total)
